# Numba 实验

现在轮到你了！

你的挑战是使用 Numba 在 CUDA Python 中实现 Fizz Buzz，并计算 `1` 到 `50_000_000` 之间所有数字的 `Fizz`、`Buzz` 和 `Fizz Buzz` 实例。

开始之前，如果你在 Google Colab 中运行此笔记本，请关闭自动补全功能：点击右上角的设置齿轮 -> Editor -> 取消勾选 "Automatically trigger code completions"。

In [1]:
import numpy as np
from numba import cuda
from numba import config as numba_config
numba_config.CUDA_ENABLE_PYNVJITLINK = True

**1. 创建输入数据数组。** 尝试使用 `np.arange`。

In [2]:
data = np.arange(1, 50000001, dtype=np.int32)
data

array([       1,        2,        3, ..., 49999998, 49999999, 50000000],
      shape=(50000000,), dtype=int32)

**2. 创建输出数据数组。**

In [3]:
output = np.zeros(len(data), dtype=np.int32)
output

array([0, 0, 0, ..., 0, 0, 0], shape=(50000000,), dtype=int32)

**3. 计算线程数和块数。**

In [4]:
threads = 128
blocks = (len(data) + threads - 1) // threads

**4. 创建内核函数。** _提示：我们的输出数组必须是数值型的，因此可以尝试使用值 `1`、`2` 和 `3` 分别表示 `Fizz`、`Buzz` 和 `Fizz Buzz`。_

In [5]:
@cuda.jit
def fizz_buzz(input_array, output_array):
    # 获取当前线程的全局索引
    i = cuda.grid(1)
    
    # 边界检查：确保不越界
    if i < input_array.size:
        num = input_array[i]
        
        # Fizz Buzz 逻辑
        if num % 15 == 0:  # 能被 15 整除（同时被 3 和 5 整除）
            output_array[i] = 3  # Fizz Buzz
        elif num % 3 == 0:  # 能被 3 整除
            output_array[i] = 1  # Fizz
        elif num % 5 == 0:  # 能被 5 整除
            output_array[i] = 2  # Buzz
        else:
            output_array[i] = 0  # 其他


**5. 运行我们的内核函数。**

In [7]:
%%time
fizz_buzz[blocks, threads](data, output)
output[:30]

CPU times: user 203 ms, sys: 0 ns, total: 203 ms
Wall time: 202 ms


array([0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
       0, 1, 2, 0, 1, 0, 0, 3], dtype=int32)

**附加题 6. 计算 `Fizz` 的最高实例。**

In [8]:
fizz_indices = np.where(output == 1)[0]
idx = fizz_indices[-1]
data[idx]

np.int32(49999998)